In [1]:
import numpy as np
import math

In [2]:
def ComputeTreeNodes(tree):    
    n = len(tree)                       # num of nodes (1, 2, 3, ...)
    h = int(math.ceil(np.log2(n+1)))    # total layers (1, 2, 3, ...)    
    seed = [1, -1, 1, -1, -1, 1, -1, 1] # calculation sign seed

    for layer in xrange(1, h):
        delta = 2**int(math.floor((layer-1)/2))
        seedPointer = 0
        for node in xrange(2**layer-1, min(n, 2**(layer+1)-1)):             
            parent = int(math.floor((node-1)/2))

            # compute row |   
            if layer%2 == 0:         
                p = tree[parent].shape[1]
                tree[node] = np.zeros((p, p), dtype=int)
                for i in xrange(0, p):            
                    tree[node][i, :] = tree[parent][i, :] + tree[parent][i+delta, :]*seed[seedPointer]
            
            # compute col ->
            else:
                p = tree[parent].shape[0]
                tree[node] = np.zeros((p, p - delta), dtype=int)

                for i in xrange(0, p - delta):
                    tree[node][:, i] = tree[parent][:, i] + tree[parent][:, i+delta]*seed[seedPointer]
        
            seedPointer = (seedPointer+1)%8

In [12]:
# Construct WHT compute tree
def WHTTree(patch, patchSize, numOfBase):    
    h = 2*np.log2(patchSize) # Levels of binary tree
    length = int(2**h-1) + numOfBase # Total length of full binary tree
    tree = np.empty(length, dtype=object)        
    tree[0] = patch
     
    ComputeTreeNodes(tree)
    return tree

In [23]:
m = 3
p = 2**m # window(patch) size
numOfBase = 16

In [8]:
#patch = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
patch = np.random.randint(255, size=(16, 16))
print(patch)

[[196 223 208 247  63 200 182 240 213 148  62 217 209  93 247 199]
 [205  23  84 156 103  15  88 175 202 103 109 148 197  24 160 185]
 [ 75 108  96   8  52 248 114 149 192  95 174  12 116 128 147  39]
 [ 38 250 208 214 214  24  54  25  88  86  63 218 220  54  37   9]
 [174  85 145 113 244  65   3 100 146 248 174  14  33  36  55 166]
 [105 119 234  17 154 151 183  77 185 113  73  11 166  78 226 236]
 [ 33 128 223 164 188   3 221  75 141 182  83 157   5 110 242 136]
 [144  69  30 141 126 178   8   4 145  61  44 126  88 179  31 216]
 [202 108 141 148 196  44 141   8 132  88  51  51  98  44 239 231]
 [245 155 245 125  86 155  57 202  29  67 236 211 219 184 194 147]
 [101  21  38 113 141 187  96 118  60 235 200  81 177  59 248 244]
 [203 188  75 182 252 238 159  47  32 165  38 169 103  82  23 121]
 [ 36  26 194   4 208 207   8 236  36  29 149 106 171  85  64 198]
 [144 129 234 136 105  71  70 252 218 130 236 223 157 146 216 207]
 [177 210 169 102 213  21 163  62 182 243 144   6 239  34 245 

In [9]:
tree = WHTTree(patch, p, numOfBase)

In [10]:
tree

array([ array([[196, 223, 208, 247,  63, 200, 182, 240, 213, 148,  62, 217, 209,
         93, 247, 199],
       [205,  23,  84, 156, 103,  15,  88, 175, 202, 103, 109, 148, 197,
         24, 160, 185],
       [ 75, 108,  96,   8,  52, 248, 114, 149, 192,  95, 174,  12, 116,
        128, 147,  39],
       [ 38, 250, 208, 214, 214,  24,  54,  25,  88,  86,  63, 218, 220,
         54,  37,   9],
       [174,  85, 145, 113, 244,  65,   3, 100, 146, 248, 174,  14,  33,
         36,  55, 166],
       [105, 119, 234,  17, 154, 151, 183,  77, 185, 113,  73,  11, 166,
         78, 226, 236],
       [ 33, 128, 223, 164, 188,   3, 221,  75, 141, 182,  83, 157,   5,
        110, 242, 136],
       [144,  69,  30, 141, 126, 178,   8,   4, 145,  61,  44, 126,  88,
        179,  31, 216],
       [202, 108, 141, 148, 196,  44, 141,   8, 132,  88,  51,  51,  98,
         44, 239, 231],
       [245, 155, 245, 125,  86, 155,  57, 202,  29,  67, 236, 211, 219,
        184, 194, 147],
       [101,  21,  38,

In [19]:
np.vstack(tree[-numOfBase:]).flatten() # The last numOfBase bases in the tree

array([2299,  197, -107, -417])

In [20]:
# Test in 800*600 image

In [27]:
import time

IMAGE_WIDTH = 800
IAMGE_HEIGHT = 600
sourceImage = np.random.randint(255, size=(IAMGE_HEIGHT, IMAGE_WIDTH))

startTime = int(round(time.time() * 1000))
#
tree = WHTTree(sourceImage, p, numOfBase)
#
endTime = int(round(time.time() * 1000))
elaps = endTime - startTime
print(elaps)

565


In [ ]:
# 4.8s perhaps
# Can be simplyfied 2**3 = 64, just compute the first(in a predefined order) 16 results

In [26]:
tree[-numOfBase:]

array([ array([[7194, 7117, 7536, 7680, 7260, 7054, 6906, 6690, 6846, 7088, 7028,
        7141, 7259],
       [7304, 7245, 7736, 8217, 7936, 7964, 7705, 7478, 7618, 7657, 7645,
        7551, 7300],
       [7560, 7397, 7998, 8309, 8135, 7876, 7548, 7053, 7468, 7493, 7333,
        7362, 6957],
       [7896, 7706, 8082, 8141, 7915, 7744, 7521, 6812, 7166, 7114, 7311,
        7319, 7025],
       [7912, 7708, 8026, 8048, 7767, 7872, 7441, 6795, 7053, 7028, 7274,
        7416, 7295],
       [8274, 8223, 8501, 8407, 8061, 8165, 7836, 7306, 7508, 7340, 7578,
        7556, 7574],
       [8386, 8293, 8559, 8494, 8119, 8030, 7876, 7608, 7786, 7616, 8076,
        7989, 8012],
       [8186, 8010, 7992, 8166, 7839, 7672, 7410, 7532, 7796, 7708, 8290,
        8256, 8459],
       [8287, 8095, 8195, 8116, 7869, 7684, 7368, 7610, 7953, 8067, 8476,
        8733, 9155],
       [8504, 8246, 8517, 8181, 7743, 7215, 7166, 7300, 7907, 8051, 8285,
        8746, 9326],
       [8739, 8674, 8888, 8637, 7963, 7832

In [ ]:
# playground

In [7]:
np.log2(5)

2.3219280948873622

In [10]:
math.floor(3.2)

3.0

In [8]:
for i in xrange(1,5):
    print(i)

1
2
3
4


In [2]:
def ComputeLeftChild(tree, parent, computeCol=True):
    index = 2*parent+1
    if index >= len(tree):
        return
    
    delta = 2**int((int(np.log2(parent+1))/2)) # level = int(log2(index+1))     
    if computeCol:
        p = tree[parent].shape[0]
        tree[index] = np.zeros((p, p - delta), dtype=int)
        for i in xrange(0, p - delta):
            tree[index][:, i] = tree[parent][:, i] - tree[parent][:, i+delta]
            
    else:
        p = tree[parent].shape[1]
        tree[index] = np.zeros((p, p), dtype=int)
        for i in xrange(0, p):
            tree[index][i, :] = tree[parent][i, :] - tree[parent][i+delta, :]

    computeCol = not computeCol
    ComputeLeftChild(tree, index, computeCol)
    ComputeRightChild(tree, index, computeCol)

In [3]:
def ComputeRightChild(tree, parent, computeCol=True):
    index = 2*parent+2
    if index >= len(tree):
        return
    
    delta = 2**int((int(np.log2(parent+1))/2)) # level = int(log2(index+1))
    if computeCol:
        p = tree[parent].shape[0]
        tree[index] = np.zeros((p, p - delta), dtype=int)
        for i in xrange(0, p - delta):
            tree[index][:, i] = tree[parent][:, i] + tree[parent][:, i+delta]
            
    else:
        p = tree[parent].shape[1]
        tree[index] = np.zeros((p, p), dtype=int)
        for i in xrange(0, p):            
            tree[index][i, :] = tree[parent][i, :] + tree[parent][i+delta, :] 
    
    computeCol = not computeCol
    ComputeLeftChild(tree, index, computeCol)
    ComputeRightChild(tree, index, computeCol)